# Chapter 9: Advanced Text Processing with Spark

* 싸이그래머, 스사모 / 스칼라ML : 파트2 - SparkML [1]
* 김무성

# Contents

* What's so special about text data?
* Extracting the right features from your data
* Using a TF-IDF model
* Evaluating the impact of text processing
* Word2Vec models
* Summary

# What's so special about text data?

* Text data can be complex to work with for two main reasons. 
    - First, text and language have an inherent structure that is not easily captured using the raw words as is (for example, meaning, context, different types of words, sentence structure, and different languages, to highlight a few). 
    - Therefore, naïve feature extraction is usually relatively ineffective.

# Extracting the right features from your data

* Term weighting schemes
* Feature hashing
* Extracting the TF-IDF features from the 20 Newsgroups dataset

#### natural language processing (NLP)

In this chapter, we will focus on two feature extraction techniques available within MLlib: the TF-IDF term weighting scheme and feature hashing.

## Term weighting schemes

* bag-of-words
* term frequency-inverse document frequency (TF-IDF)
* document
* inverse document frequency
* corpus

<img src="figures/cap9.1.png" />

<img src="figures/cap9.2.png" />

## Feature hashing

Feature hashing is a technique to deal with high-dimensional data and is often used with text and categorical datasets where the features can take on many unique values (often many millions of values). 

* 1-of-K feature encoding
* hashing

However, there are two important drawbacks, which are as follows:

* As we don't create a mapping of features to index values, we also cannot do the reverse mapping of feature index to value. This makes it harder to, for example, determine which features are most informative in our models.
* As we are restricting the size of our feature vectors, we might experience hash collisions. This happens when two different features are hashed into the same index in our feature vector. Surprisingly, this doesn't seem to have a severe impact on model performance as long as we choose a reasonable feature vector dimension relative to the dimension of the input data.

## Extracting the TF-IDF features from the 20 Newsgroups dataset

* Exploring the 20 Newsgroups data
* Applying basic tokenization
* Improving our tokenization
* Removing stop words
* Excluding terms based on frequency
* A note about stemming
* Training a TF-IDF model 
* Analyzing the TF-IDF weightings

To illustrate the concepts in this chapter, we will use a well-known text dataset called 20 Newsgroups; this dataset is commonly used for text-classification tasks. 

This dataset splits up the available data into training and test sets that comprise
60 percent and 40 percent of the original data, respectively. 

In [ ]:
!wget http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz 

In [ ]:
!tar xfvz 20news-bydate.tar.gz

In [1]:
!ls

20news-bydate-test   20news-bydate.tar.gz			  figures
20news-bydate-train  9_Advanced_Text_Processing_with_Spark.ipynb


In [2]:
!ls 20news-bydate-train/  

alt.atheism		  rec.autos	      sci.space
comp.graphics		  rec.motorcycles     soc.religion.christian
comp.os.ms-windows.misc   rec.sport.baseball  talk.politics.guns
comp.sys.ibm.pc.hardware  rec.sport.hockey    talk.politics.mideast
comp.sys.mac.hardware	  sci.crypt	      talk.politics.misc
comp.windows.x		  sci.electronics     talk.religion.misc
misc.forsale		  sci.med


There are a number of files under each newsgroup folder; each file contains an individual message posting:

In [3]:
!ls 20news-bydate-train/rec.sport.hockey

52550  52605  52660  53565  53620  53675  53731  53786	53841  53905  54058
52551  52606  52661  53566  53621  53676  53732  53787	53842  53906  54070
52552  52607  52662  53567  53622  53677  53733  53788	53843  53907  54071
52553  52608  52663  53568  53623  53678  53734  53789	53844  53908  54076
52554  52609  52664  53569  53624  53679  53735  53790	53845  53909  54079
52555  52610  52665  53570  53625  53680  53736  53791	53846  53910  54080
52556  52611  52666  53571  53626  53681  53737  53792	53847  53911  54094
52557  52612  52667  53572  53627  53682  53738  53793	53848  53912  54113
52558  52613  52668  53573  53628  53683  53739  53794	53849  53913  54117
52559  52614  52669  53574  53629  53684  53740  53795	53850  53914  54118
52560  52615  53468  53575  53630  53685  53741  53796	53851  53915  54122
52561  52616  53521  53576  53631  53686  53742  53797	53852  53916  54123
52562  52617  53522  53577  53632  53687  53743  53798	53853  53917  54124
52563  52618  53523  5357

In [4]:
!head -20 20news-bydate-train/rec.sport.hockey/52550

From: dchhabra@stpl.ists.ca (Deepak Chhabra)
Subject: Superstars and attendance (was Teemu Selanne, was +/- leaders)
Nntp-Posting-Host: stpl.ists.ca
Organization: Solar Terresterial Physics Laboratory, ISTS
Distribution: na
Lines: 115


Dean J. Falcione (posting from jrmst+8@pitt.edu) writes:
[I wrote:]

>>When the Pens got Mario, granted there was big publicity, etc, etc,
>>and interest was immediately generated.  Gretzky did the same thing for LA. 
>>However, imnsho, neither team would have seen a marked improvement in
>>attendance if the team record did not improve.  In the year before Lemieux
>>came, Pittsburgh finished with 38 points.  Following his arrival, the Pens
>>finished with 53, 76, 72, 81, 87, 72, 88, and 87 points, with a couple of
                          ^^
>>Stanley Cups thrown in.
      


### Exploring the 20 Newsgroups data

we will again use Spark's wholeTextFiles method to read the content of each file into a record in our RDD.

you will see the total record count, which should be the same as the preceding Total input paths to process screen output:

In [3]:
val path = "./20news-bydate-train/*"
val rdd = sc.wholeTextFiles(path)
val text = rdd.map { case (file, text) => text }
println(text.count)

11314


Next, we will take a look at the newsgroup topics available:

We can see that the number of messages is roughly even between the topics.

In [4]:
val newsgroups = rdd.map{ case (file, text) => file.split("/").takeRight(2).head }
val countByGroup = newsgroups.map(n => (n, 1)).reduceByKey(_ + _).collect.sortBy(-_._2).mkString("\n")
println(countByGroup)

(rec.sport.hockey,600)
(soc.religion.christian,599)
(rec.motorcycles,598)
(rec.sport.baseball,597)
(sci.crypt,595)
(sci.med,594)
(rec.autos,594)
(sci.space,593)
(comp.windows.x,593)
(sci.electronics,591)
(comp.os.ms-windows.misc,591)
(comp.sys.ibm.pc.hardware,590)
(misc.forsale,585)
(comp.graphics,584)
(comp.sys.mac.hardware,578)
(talk.politics.mideast,564)
(talk.politics.guns,546)
(alt.atheism,480)
(talk.politics.misc,465)
(talk.religion.misc,377)


### Applying basic tokenization

* tokens
* tokenization
* whitespace

We will start by applying a simple whitespace tokenization, together with converting each token to lowercase for each document:

After running this code snippet, you will see the total number of unique tokens after applying our tokenization:

In [5]:
val text = rdd.map{ case (file, text) => text }
val whiteSpaceSplit = text.flatMap(t => t.split(" ").map(_.toLowerCase))
println(whiteSpaceSplit.distinct.count)

402978


Let's take a look at a randomly selected document:

In [6]:
println(whiteSpaceSplit.sample(true, 0.3, 42).take(100).mkString(","))

from:,jar2e@faraday.clas.virginia.edu,jar2e@faraday.clas.virginia.edu,re:,re:,israeli,terrorism
organization:,virginia
lines:,12

would,document,reporters"?,i,you,with,region,which,by,by,government,,be,the,the,human,human,rights,by
sealing,off,strip,,get,the,palestinian-on-palestinian
civil,and,behave,peace.,peace.,not,murtezaoglu)
subject:,re:,it,could,shoot,henrik@quayle.kpc.com's,1993,1993,16:45:17,computer,science,science,article,henrik@quayle.kpc.com,to,drag,drag,azerbaijan.,a,a,capital,not,,not,,above,that,short,that,stop,for,anyone,it
is,

>the,have,from,given,azeris,stalin),are,are,conflict.,are,defending,defending,expect,forces,them
within,their,that,turkey,turkey,
>,crisis,occur,not,playing,a,deck,,,invade?
are,the,with,header
in,hopes,greek


### Improving our tokenization

We can do this by splitting each raw document on nonword characters using a regular expression pattern:

This reduces the number of unique tokens significantly:

In [7]:
val nonWordSplit = text.flatMap(t => t.split("""\W+""").map(_.toLowerCase))
println(nonWordSplit.distinct.count)

130126


If we inspect the first few tokens, we will see that we have eliminated most of the less useful characters in the text:

In [8]:
println(nonWordSplit.distinct.sample(true, 0.3, 42).take(100).mkString(","))

kv07,jejones,jejones,bone,bone,k29p,ml5,ratifi,valuemask,bruns,mmejv5,jsoh,bluffing,125215,p05,isgal,kjd0,kjd0,c1381,200649,pacified,itchy,ishbel,ishbel,c1,c5ckp9,gustafson,nonmeasurable,1qsj1d9,q9w,omnimovie,agm,personify,framestores,framestores,salvageable,10011100b,bippy,dolphin,102756,margitan,wp3d,cannibal,cannibal,bronfman,prescient,211053,211053,committeewoman,pmjh,jesuit,cscx_sy_,cscx_sy_,incomparable,6097,6097,204843,busbar,busbar,6jx,623,dbuck,nixdorf,omputers,eavesdroppers,arely,mortal,springer,perversity,interfere,nowadays,formac,maxscreens,khb,khb,fuenfzig,paradijs,cyclops,cyclops,sx,xtaddcallback,2_patch_version3,projector,1qmrdd,65e90h8y,65e90h8y,dib,rint69,antena,mcgiver,5297,kmv6snl,xwlr3hi,kipling,adventists,lanman,9mm,0hnz,lama,dxb132


We can do this by applying another regular expression pattern and using this to filter out tokens that do not match the pattern:

This further reduces the size of the token set:

In [9]:
val regex = """[^0-9]*""".r
val filterNumbers = nonWordSplit.filter(token =>regex.pattern.matcher(token).matches)
println(filterNumbers.distinct.count)

84912


Let's take a look at another random sample of the filtered tokens:

In [10]:
println(filterNumbers.distinct.sample(true, 0.3, 42).take(100).mkString(","))

jejones,hif,hif,glorifying,glorifying,wuair,feh,schwabam,valuemask,tough,_congressional,interlaced,husky,relieves,artur,entitlements,arax,arax,phillips,mirosoft,dwi,believiing,odf,odf,steaminess,pacified,hizbolah,wqod,adultery,urtfi,nauseam,ishbel,wout,emerich,emerich,seetex,viewed,milking,museum,eur,typeset,smits,twarda,twarda,salvageable,mget,sctc,sctc,bippy,cities,mmg,root_iden,root_iden,cannibal,michaelr,michaelr,mswin,gundry,gundry,diccon,babied,belated,borg,sham,trivialized,jlange,impute,octopi,seeing,dtn,kruislaan,awdprime,detergent,yan,yan,agenzia,icbz,volcanic,volcanic,conspiricy,chov,deadweight,gcs,ouzq,kjiv,kjiv,hour,caramate,dchhabra,springer,alchoholic,cherylm,perversity,chq,bruncati,interfere,amro,exhausting,murdering,khb


### Removing stop words

* Stop words
    - Examples of typical English stop words include and, but, the, of, and so on.

We can take a look at some of the tokens in our corpus that have the highest occurrence across all documents to get an idea about some other stop words to exclude:

In the code, we took the tokens after filtering out numeric characters and generated a count of the occurrence of each token across the corpus. We can now use Spark's top function to retrieve the top 20 tokens by count.

In [11]:
val tokenCounts = filterNumbers.map(t => (t, 1)).reduceByKey(_ + _)
val oreringDesc = Ordering.by[(String, Int), Int](_._2)
println(tokenCounts.top(20)(oreringDesc).mkString("\n"))

(the,146532)
(to,75064)
(of,69034)
(a,64195)
(ax,62406)
(and,57957)
(i,53036)
(in,49402)
(is,43480)
(that,39264)
(it,33638)
(for,28600)
(you,26682)
(from,22670)
(s,22337)
(edu,21321)
(on,20493)
(this,20121)
(be,19285)
(t,18728)


As we might expect, there are a lot of common words in this list that we could potentially label as stop words. Let's create a set of stop words with some of these as well as other common words. We will then look at the tokens after filtering out these stop words:

In [12]:
val stopwords = Set(
     "the","a","an","of","or","in","for","by","on","but", "is", "not",
   "with", "as", "was", "if",
     "they", "are", "this", "and", "it", "have", "from", "at", "my",
   "be", "that", "to"
   )
val tokenCountsFilteredStopwords = tokenCounts.filter { case(k, v) => !stopwords.contains(k) }
println(tokenCountsFilteredStopwords.top(20)(oreringDesc).mkString("\n"))

(ax,62406)
(i,53036)
(you,26682)
(s,22337)
(edu,21321)
(t,18728)
(m,12756)
(subject,12264)
(com,12133)
(lines,11835)
(can,11355)
(organization,11233)
(re,10534)
(what,9861)
(there,9689)
(x,9332)
(all,9310)
(will,9279)
(we,9227)
(one,9008)


One other filtering step that we will use is removing any tokens that are only one character in length. 

In [13]:
val tokenCountsFilteredSize = tokenCountsFilteredStopwords.filter{ case (k, v) => k.size >= 2 }
println(tokenCountsFilteredSize.top(20)(oreringDesc).mkString("\n"))

(ax,62406)
(you,26682)
(edu,21321)
(subject,12264)
(com,12133)
(lines,11835)
(can,11355)
(organization,11233)
(re,10534)
(what,9861)
(there,9689)
(all,9310)
(will,9279)
(we,9227)
(one,9008)
(would,8905)
(do,8674)
(he,8441)
(about,8336)
(writes,7844)


### Excluding terms based on frequency

It is also a common practice to exclude terms during tokenization when their overall occurrence in the corpus is very low. For example, let's examine the least occurring terms in the corpus (notice the different ordering we use here to return the results sorted in ascending order):

In [14]:
val oreringAsc = Ordering.by[(String, Int), Int](-_._2)
println(tokenCountsFilteredSize.top(20)(oreringAsc).mkString("\n"))

(tenex,1)
(beckmans,1)
(wuair,1)
(feh,1)
(ratifi,1)
(schwabam,1)
(bruns,1)
(swith,1)
(bluffing,1)
(hif,1)
(actu,1)
(adnd,1)
(wbp,1)
(bunuel,1)
(uncompression,1)
(mxh,1)
(_congressional,1)
(fowl,1)
(lennips,1)
(jsoh,1)


As we can see, there are many terms that only occur once in the entire corpus. Since typically we want to use our extracted features for other tasks such as document similarity or machine learning models, tokens that only occur once are not useful to learn from, as we will not have enough training data relative to these tokens. We can apply another filter to exclude these rare tokens:

In [15]:
val rareTokens = tokenCounts.filter{ case (k, v) => v < 2 }.map {case (k, v) => k }.collect.toSet
val tokenCountsFilteredAll = tokenCountsFilteredSize.filter { case(k, v) => !rareTokens.contains(k) }
println(tokenCountsFilteredAll.top(20)(oreringAsc).mkString("\n"))

(mmg,2)
(wexler,2)
(theoreticians,2)
(glorifying,2)
(michaelr,2)
(relieves,2)
(_lwo,2)
(isgal,2)
(prescient,2)
(eoeun,2)
(mswin,2)
(kielbasa,2)
(bronfman,2)
(defiance,2)
(contoler,2)
(omnimovie,2)
(disobeyers,2)
(historicity,2)
(congresswoman,2)
(relatifs,2)


Now, let's count the number of unique tokens:

In [16]:
println(tokenCountsFilteredAll.count)

51801


We can now combine all our filtering logic into one function, which we can apply to each document in our RDD:

In [17]:
def tokenize(line: String): Seq[String] = {
     line.split("""\W+""")
       .map(_.toLowerCase)
       .filter(token => regex.pattern.matcher(token).matches)
       .filterNot(token => stopwords.contains(token))
       .filterNot(token => rareTokens.contains(token))
       .filter(token => token.size >= 2)
       .toSeq
}

We can check whether this function gives us the same result with the following code snippet:

In [18]:
println(text.flatMap(doc => tokenize(doc)).distinct.count)

51801


We can tokenize each document in our RDD as follows:

In [19]:
val tokens = text.map(doc => tokenize(doc))
println(tokens.first.take(20))

WrappedArray(faraday, clas, virginia, edu, virginia, gentleman, subject, re, israeli, terrorism, organization, university, virginia, lines, would, asking, too, much, you, document)


### A note about stemming

* A common step in text processing and tokenization is stemming. This is the conversion of whole words to a base form (called a word stem). 
    - stemming
    - base form
    - word stem
* For example, plurals might be converted to singular (dogs becomes dog), and forms such as walking and walker might become walk. 
* Stemming can become quite complex and is typically handled with specialized NLP or search engine software 
    - such as 
        - NLTK, 
        - OpenNLP, and 
        - Lucene, 
* We will ignore stemming for the purpose of our example here.

### Training a TF-IDF model 

We will now use MLlib to transform each document, in the form of processed tokens, into a vector representation. 

* The first step will be to use the HashingTF implementation, 
    which makes use of feature hashing to map each token in the input text to an index in the vector of term frequencies.
* Then, we will compute the global IDF and 
* use it to transform the term frequency vectors into TF-IDF vectors.

First, we will import the classes we need and create our HashingTF instance, passing in a dim dimension parameter. While the default feature dimension is 2^20 (or around 1 million), we will choose 2^18 (or around 260,000), since with about 50,000 tokens, we should not experience a significant number of hash collisions, and a smaller dimension will be more memory and processing friendly for illustrative purposes:

In [20]:
import org.apache.spark.mllib.linalg.{ SparseVector => SV }
import org.apache.spark.mllib.feature.HashingTF
import org.apache.spark.mllib.feature.IDF

In [21]:
val dim = math.pow(2, 18).toInt
val hashingTF = new HashingTF(dim)
val tf = hashingTF.transform(tokens)
tf.cache

MapPartitionsRDD[45] at map at HashingTF.scala:78

* The transform function of HashingTF maps each input document (that is, a sequence of tokens) to an MLlib Vector. 
* We will also call cache to pin the data in memory to speed up subsequent operations.

Let's inspect the first element of our transformed dataset:

In [22]:
val v = tf.first.asInstanceOf[SV]
println(v.size)
println(v.values.size)
println(v.values.take(10).toSeq)
println(v.indices.take(10).toSeq)

262144
67
WrappedArray(1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0)
WrappedArray(3635, 5740, 9972, 19841, 24877, 29853, 33239, 39411, 47026, 49378)


* We can see that the dimension of each sparse vector of term frequencies is 262,144 (or 218 as we specified). 
* However, the number on non-zero entries in the vector is only 706. 
* The last two lines of the output show the frequency counts and vector indexes for the first few entries in the vector.

We will now compute the inverse document frequency for each term in the corpus by creating a new IDF instance and calling fit with our RDD of term frequency vectors as the input. 

We will then transform our term frequency vectors to TF-IDF vectors through the transform function of IDF:


In [23]:
val idf = new IDF().fit(tf)
val tfidf = idf.transform(tf)
val v2 = tfidf.first.asInstanceOf[SV]
println(v2.values.size)
println(v2.values.take(10).toSeq)
println(v2.indices.take(10).toSeq)

67
WrappedArray(0.3723902347580378, 2.736738856180986, 4.891233301577321, 5.462683547159747, 6.435984865169208, 6.561295835827856, 3.414990703794491, 3.9725923923582127, 5.807524033451476, 6.038047692063309)
WrappedArray(3635, 5740, 9972, 19841, 24877, 29853, 33239, 39411, 47026, 49378)


* We can see that the number of non-zero entries hasn't changed (at 706), nor have the vector indices for the terms.

### Analyzing the TF-IDF weightings

Next, let's investigate the TF-IDF weighting for a few terms to illustrate the impact of the commonality or rarity of a term.

First, we can compute the minimum and maximum TF-IDF weights across the entire corpus:

In [31]:
val minMaxVals = tfidf.map { v =>
     val sv = v.asInstanceOf[SV]
     (sv.values.min, sv.values.max)
}
val globalMinMax = minMaxVals.reduce { case ((min1, max1),
   (min2, max2)) =>
     (math.min(min1, min2), math.max(max1, max2))
}
println(globalMinMax)

(0.0,66155.39470409753)


* As we can see, the minimum TF-IDF is zero, while the maximum is significantly larger:

TF-IDF weighting will tend to assign a lower weighting to common terms. To see this, we can compute the TF-IDF representation for a few of the terms that appear in the list of top occurrences that we previously computed, such as you, do, and we:

In [32]:
val common = sc.parallelize(Seq(Seq("you", "do", "we")))
val tfCommon = hashingTF.transform(common)
val tfidfCommon = idf.transform(tfCommon)
val commonVector = tfidfCommon.first.asInstanceOf[SV]
println(commonVector.values.toSeq)

WrappedArray(0.9965359935704624, 1.3348773448236835, 0.5457486182039175)


* If we form a TF-IDF vector representation of this document, we would see the values assigned to each term. Note that because of feature hashing, we are not sure exactly which term represents what. However, the values illustrate that the weighting applied to these terms is relatively low:

Now, let's apply the same transformation to a few less common terms that we might intuitively associate with being more linked to specific topics or concepts:

In [33]:
val uncommon = sc.parallelize(Seq(Seq("telescope", "legislation", "investment")))
val tfUncommon = hashingTF.transform(uncommon)
val tfidfUncommon = idf.transform(tfUncommon)
val uncommonVector = tfidfUncommon.first.asInstanceOf[SV]
println(uncommonVector.values.toSeq)

WrappedArray(5.3265513728351666, 5.308532867332488, 5.483736956357579)


# Using a TF-IDF model

* Document similarity with the 20 Newsgroups dataset and TF-IDF features
* Training a text classifier on the 20 Newsgroups dataset using TF-IDF

TF-IDF weighting is often used as a preprocessing step for other models, such as dimensionality reduction, classification, or regression.

To illustrate the potential uses of TF-IDF weighting, we will explore two examples. 

* The first is using the TF-IDF vectors to compute document similarity, 
* while the second involves training a multilabel classification model with the TF-IDF vectors as input features.

## Document similarity with the 20 Newsgroups dataset and TF-IDF features

Intuitively, we might expect two documents to be more similar to each other if they share many terms. Conversely, we might expect two documents to be less similar
if they each contain many terms that are different from each other. As we compute cosine similarity by computing a dot product of the two vectors and each vector
is made up of the terms in each document, we can see that documents with a high overlap of terms will tend to have a higher cosine similarity.

For example, we might expect two randomly chosen messages from the hockey newsgroup to be relatively similar to each other. Let's see if this is the case:

In [34]:
val hockeyText = rdd.filter { case (file, text) => file.contains("hockey") }
val hockeyTF = hockeyText.mapValues(doc =>hashingTF.transform(tokenize(doc)))
val hockeyTfIdf = idf.transform(hockeyTF.map(_._2))

Once we have our hockey document vectors, we can select two of these vectors at random and compute the cosine similarity between them (as we did earlier, we will use Breeze for the linear algebra functionality, in particular converting our MLlib vectors to Breeze SparseVector instances first):

In [35]:
import breeze.linalg._
val hockey1 = hockeyTfIdf.sample(true, 0.1, 42).first.asInstanceOf[SV]
val breeze1 = new SparseVector(hockey1.indices, hockey1.values,hockey1.size)
val hockey2 = hockeyTfIdf.sample(true, 0.1, 43).first.asInstanceOf[SV]
val breeze2 = new SparseVector(hockey2.indices, hockey2.values,hockey2.size)
val cosineSim = breeze1.dot(breeze2) / (norm(breeze1) * norm(breeze2))
println(cosineSim)

0.05028503626731515


By contrast, we can compare this similarity score to the one computed between one of our hockey documents and another document chosen randomly from the comp.graphics newsgroup, using the same methodology:

In [36]:
val graphicsText = rdd.filter { case (file, text) => file.contains("comp.graphics") }
val graphicsTF = graphicsText.mapValues(doc => hashingTF.transform(tokenize(doc)))
val graphicsTfIdf = idf.transform(graphicsTF.map(_._2))
val graphics = graphicsTfIdf.sample(true, 0.1, 42).first.asInstanceOf[SV]
val breezeGraphics = new SparseVector(graphics.indices,graphics.values, graphics.size)
val cosineSim2 = breeze1.dot(breezeGraphics) / (norm(breeze1) * norm(breezeGraphics))
println(cosineSim2)

0.008518345400584703


Finally, it is likely that a document from another sports-related topic might be more similar to our hockey document than one from a computer-related topic. However, we would probably expect a baseball document to not be as similar as our hockey document. Let's see whether this is the case by computing the similarity between a random message from the baseball newsgroup and our hockey document:

In [37]:
val baseballText = rdd.filter { case (file, text) => file.contains("baseball") }
val baseballTF = baseballText.mapValues(doc => hashingTF.transform(tokenize(doc)))
val baseballTfIdf = idf.transform(baseballTF.map(_._2))
val baseball = baseballTfIdf.sample(true, 0.1, 42).first.asInstanceOf[SV]
val breezeBaseball = new SparseVector(baseball.indices,baseball.values, baseball.size)
val cosineSim3 = breeze1.dot(breezeBaseball) / (norm(breeze1) * norm(breezeBaseball))
println(cosineSim3)

0.006477216225017649


* Indeed, as we expected, we found that the baseball and hockey documents have a cosine similarity of 0.05, which is significantly higher than the comp.graphics document, but also somewhat lower than the other hockey document:

## Training a text classifier on the 20 Newsgroups dataset using TF-ID

When using TF-IDF vectors, we expected that the cosine similarity measure would capture the similarity between documents, based on the overlap of terms between them. In a similar way, we would expect that a machine learning model, such as a classifier, would be able to learn weightings for individual terms; this would allow it to distinguish between documents from different classes. That is, it should be possible to learn a mapping between the presence (and weighting) of certain terms and a specific topic.

In the 20 Newsgroups example, each newsgroup topic is a class, and we can train a classifier using our TF-IDF transformed vectors as input.


Since we are dealing with a multiclass classification problem, we will use the naïve Bayes model in MLlib, which supports multiple classes. As the first step, we will import the Spark classes that we will be using:

In [38]:
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.classification.NaiveBayes
import org.apache.spark.mllib.evaluation.MulticlassMetrics

Next, we will need to extract the 20 topics and convert them to class mappings. We can do this in exactly the same way as we might for 1-of-K feature encoding, by assigning a numeric index to each class:

In [39]:
val newsgroupsMap = newsgroups.distinct.collect().zipWithIndex.toMap
val zipped = newsgroups.zip(tfidf)
val train = zipped.map { case (topic, vector) => LabeledPoint(newsgroupsMap(topic), vector) }
train.cache

MapPartitionsRDD[80] at map at <console>:61

* In the preceding code snippet, we took the newsgroups RDD, where each element is the topic, and used the zip function to combine it with each element in our tfidf RDD of TF-IDF vectors.
* We then mapped over each key-value element in our new zipped RDD and created a LabeledPoint instance, where label is the class index and features is the TF-IDF vector.

Now that we have an input RDD in the correct form, we can simply pass it to the naïve Bayes train function:

In [40]:
val model = NaiveBayes.train(train, lambda = 0.1)

Let's evaluate the performance of the model on the test dataset. We will load the raw test data from the 20news-bydate-test directory, again using wholeTextFiles to read each message into an RDD element. We will then extract the class labels from the file paths in the same way as we did for the newsgroups RDD:

In [41]:
val testPath = "./20news-bydate-test/*"
val testRDD = sc.wholeTextFiles(testPath)
val testLabels = testRDD.map { case (file, text) => 
    val topic = file.split("/").takeRight(2).head
    newsgroupsMap(topic)
}

In [42]:
val testTf = testRDD.map { case (file, text) => hashingTF.transform(tokenize(text)) }
val testTfIdf = idf.transform(testTf)
val zippedTest = testLabels.zip(testTfIdf)
val test = zippedTest.map { case (topic, vector) => LabeledPoint(topic, vector) }

In [1]:
val predictionAndLabel = test.map(p => (model.predict(p.features),p.label))
val accuracy = 1.0 * predictionAndLabel.filter(x => x._1 == x._2).count() / test.count()
val metrics = new MulticlassMetrics(predictionAndLabel)
println(accuracy)
println(metrics.weightedFMeasure)

Name: Compile Error
Message: <console>:12: error: not found: value test
       val predictionAndLabel = test.map(p => (model.predict(p.features),p.label))
                                ^
StackTrace: 

# Evaluating the impact of text processing

* Comparing raw features with processed TF-IDF features on the 20 Newsgroups dataset

We can see the impact of applying these processing techniques by comparing the performance of a model trained on raw text data with one trained on processed and TF-IDF weighted text data.

## Comparing raw features with processed TF-IDF features on the 20 Newsgroups dataset

In this example, we will simply apply the hashing term frequency transformation to the raw text tokens obtained using a simple whitespace splitting of the document text. We will train a model on this data and evaluate the performance on the test set as we did for the model trained with TF-IDF features:

In [ ]:
val rawTokens = rdd.map { case (file, text) => text.split(" ") }
val rawTF = texrawTokenst.map(doc => hashingTF.transform(doc))
val rawTrain = newsgroups.zip(rawTF).map { case (topic, vector) =>
   LabeledPoint(newsgroupsMap(topic), vector) }
val rawModel = NaiveBayes.train(rawTrain, lambda = 0.1)
val rawTestTF = testRDD.map { case (file, text) =>
   hashingTF.transform(text.split(" ")) }
val rawZippedTest = testLabels.zip(rawTestTF)
val rawTest = rawZippedTest.map { case (topic, vector) =>
   LabeledPoint(topic, vector) }
val rawPredictionAndLabel = rawTest.map(p =>
   (rawModel.predict(p.features), p.label))
val rawAccuracy = 1.0 * rawPredictionAndLabel.filter(x => x._1 ==
   x._2).count() / rawTest.count()
println(rawAccuracy)
val rawMetrics = new MulticlassMetrics(rawPredictionAndLabel)
println(rawMetrics.weightedFMeasure)

*  This is also partly a reflection of the fact that the naïve Bayes model is well suited to data in the form of raw frequency counts:

# Word2Vec models

* Word2Vec on the 20 Newsgroups dataset

#### distributed vector representations

* Word2Vec refers to a specific implementation of one of these models, often referred to as distributed vector representations. 
* The MLlib model uses a skip-gram model, which seeks to learn vector representations that take into account the contexts in which words occur.

## Word2Vec on the 20 Newsgroups dataset

Training a Word2Vec model in Spark is relatively simple. We will pass in an RDD where each element is a sequence of terms. We can use the RDD of tokenized documents we have already created as input to the model:

In [ ]:
import org.apache.spark.mllib.feature.Word2Vec
val word2vec = new Word2Vec()
word2vec.setSeed(42)
val word2vecModel = word2vec.fit(tokens)

Once trained, we can easily find the top 20 synonyms for a given term (that is, the most similar term to the input term, computed by cosine similarity between the word vectors). For example, to find the 20 most similar terms to hockey, use the following lines of code:

In [ ]:
word2vecModel.findSynonyms("hockey", 20).foreach(println)

In [ ]:
word2vecModel.findSynonyms("legislation", 20).foreach(println)

# Summary

# 참고자료

* [1] book - https://www.packtpub.com/big-data-and-business-intelligence/machine-learning-spark
* [2] jypyter/all-spark-notebook docker - https://hub.docker.com/r/jupyter/all-spark-notebook/